In [1]:
!pip install torch_geometric

In [2]:
!pip install torch-scatter==latest+cu101 torch-sparse==latest+cu101 -f https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0.html

Looking in links: https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0.html
  Using cached https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0/torch_scatter-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Using cached https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0/torch_sparse-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch-scatter 2.0.4
    Uninstalling torch-scatter-2.0.4:
      Successfully uninstalled torch-scatter-2.0.4
  Found existing installation: torch-sparse 0.6.0
    Uninstalling torch-sparse-0.6.0:
      Successfully uninstalled torch-sparse-0.6.0


In [0]:
%pwd

'/content'

In [0]:
import torch
import torch.nn as nn
from sklearn.metrics import f1_score
import numpy as np
import torch.nn.functional as F
import time

In [0]:
import sys
sys.path.insert(0, '/content/drive/My Drive')

In [0]:
from my_utils import *

In [1]:
train_node_ratio = 44906. / 56944
val_node_ratio = 6514. / 56944

In [7]:
data = load_PPI_for_transductive_learning(train_node_ratio, val_node_ratio, 2, shuffle=False)

运行很顺畅！


In [8]:
 list(data['train_loader'])

[Data(edge_index=[2, 65166], mask=[3144], x=[3144, 50], y=[3144, 121]),
 Data(edge_index=[2, 129676], mask=[4602], x=[4602, 50], y=[4602, 121]),
 Data(edge_index=[2, 56977], mask=[2599], x=[2599, 50], y=[2599, 121]),
 Data(edge_index=[2, 119031], mask=[4311], x=[4311, 50], y=[4311, 121]),
 Data(edge_index=[2, 117809], mask=[3903], x=[3903, 50], y=[3903, 121]),
 Data(edge_index=[2, 114765], mask=[4279], x=[4279, 50], y=[4279, 121]),
 Data(edge_index=[2, 157821], mask=[5299], x=[5299, 50], y=[5299, 121]),
 Data(edge_index=[2, 150300], mask=[5120], x=[5120, 50], y=[5120, 121]),
 Data(edge_index=[2, 168049], mask=[5465], x=[5465, 50], y=[5465, 121]),
 Data(edge_index=[2, 191680], mask=[6184], x=[6184, 50], y=[6184, 121]),
 Data(edge_index=[2, 205434], mask=[6514], x=[6514, 50], y=[6514, 121]),
 Data(edge_index=[2, 167500], mask=[5524], x=[5524, 50], y=[5524, 121])]

In [0]:
train_loader = list(data['train_loader'])
train_loader = [train_loader[2]]
val_loader = list(data['val_loader'])
val_loader = [val_loader[2]]
test_loader = list(data['test_loader'])
test_loader = [test_loader[2]]

In [10]:
test_loader

[Data(edge_index=[2, 56977], mask=[2599], x=[2599, 50], y=[2599, 121])]

In [0]:
class BCEWithLogitsWrapper(nn.Module):
    def __init__(self):
        super(BCEWithLogitsWrapper, self).__init__()
        self.loss = nn.BCEWithLogitsLoss()

    def forward(self, logits, labels):
        return self.loss(logits, labels.float())

In [0]:
from torch_geometric.nn import GATConv

In [0]:
from utils import EarlyStopping

In [0]:
num_features = 50
num_classes = 121

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GATConv(num_features, 256, heads=4)
        self.lin1 = torch.nn.Linear(num_features, 4 * 256)
        self.conv2 = GATConv(4 * 256, 256, heads=4)
        self.lin2 = torch.nn.Linear(4* 256, 4 * 256)
        self.conv3 = GATConv(
            4 * 256, num_classes, heads=4, concat=False)
        self.lin3 = torch.nn.Linear(4* 256, num_classes)

    def forward(self, x, edge_index):
        #x, edge_index = data.x, data.edge_index
        #x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index) + self.lin1(x))
        #x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv2(x, edge_index) + self.lin2(x))
        x = self.conv3(x, edge_index) + self.lin3(x)
        return x

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
#loss_op = torch.nn.BCEWithLogitsLoss()
loss_op = BCEWithLogitsWrapper()
#optimizer = torch.optim.Adam(model.parameters(), lr=0.03, weight_decay=5e-6)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [0]:

gats = []
for _ in range(10):
  batch_size = 2
  def train():
      model.train()

      total_loss = 0

      for dat in train_loader:
          num_graphs = batch_size
          
          #dat.batch = None
          dat = dat.to(device)
          optimizer.zero_grad()

          outs = model(dat.x, dat.edge_index)
          loss = loss_op(outs[dat.mask], dat.y[dat.mask])
          temp = loss.item() * num_graphs
          total_loss += temp
          loss.backward()
          optimizer.step()
      return total_loss /2


  def test(loader):
      model.eval()

      ys, preds = [], []
      for dat in loader:
          ys.append(dat.y[dat.mask])
          with torch.no_grad():
              out = model(dat.x.to(device), dat.edge_index.to(device))[dat.mask]
          preds.append((out > 0).float().cpu())

      y, pred = torch.cat(ys, dim=0).numpy(), torch.cat(preds, dim=0).numpy()
      return f1_score(y, pred, average='micro') if pred.sum() > 0 else 0


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []

  print(model)
  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_f1 = test(val_loader)
      
      if early_stop:
          if stopper.step(val_f1, model):   
              break
            
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_f1 = test(test_loader)
  gats.append(test_f1)
  print("Test Accuracy {:.4f}".format(test_f1))
    

Net(
  (conv1): GATConv(50, 256, heads=4)
  (lin1): Linear(in_features=50, out_features=1024, bias=True)
  (conv2): GATConv(1024, 256, heads=4)
  (lin2): Linear(in_features=1024, out_features=1024, bias=True)
  (conv3): GATConv(1024, 121, heads=4)
  (lin3): Linear(in_features=1024, out_features=121, bias=True)
)
Test Accuracy 0.6463
Net(
  (conv1): GATConv(50, 256, heads=4)
  (lin1): Linear(in_features=50, out_features=1024, bias=True)
  (conv2): GATConv(1024, 256, heads=4)
  (lin2): Linear(in_features=1024, out_features=1024, bias=True)
  (conv3): GATConv(1024, 121, heads=4)
  (lin3): Linear(in_features=1024, out_features=121, bias=True)
)
Test Accuracy 0.6493
Net(
  (conv1): GATConv(50, 256, heads=4)
  (lin1): Linear(in_features=50, out_features=1024, bias=True)
  (conv2): GATConv(1024, 256, heads=4)
  (lin2): Linear(in_features=1024, out_features=1024, bias=True)
  (conv3): GATConv(1024, 121, heads=4)
  (lin3): Linear(in_features=1024, out_features=121, bias=True)
)
Test Accuracy 0.

In [0]:
np.round(gats,3).tolist()

[0.646, 0.649, 0.649, 0.659, 0.658, 0.658, 0.658, 0.657, 0.658, 0.666]

In [0]:
np.mean(gats)

0.6558688047363785

GCN

In [0]:
from torch_geometric.nn import GCNConv

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(num_features, 4* 256, cached=True)
       
        self.conv2 = GCNConv(4* 256, 4 * 256, cached=True)
        
        self.conv3 = GCNConv(4 * 256, num_classes, cached=True)
        
        
    def forward(self, x, edge_index):
        #x = F.dropout(x, training=self.training)
        x = F.relu(self.conv1(x, edge_index))
       
        x = F.relu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)
        return x

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
loss_op = BCEWithLogitsWrapper()
#optimizer = torch.optim.Adam(model.parameters(), lr=0.03)
optimizer = torch.optim.Adam(model.parameters(), lr=0.03, weight_decay=5e-6)

In [0]:

gcns = []
for _ in range(10):
  batch_size = 2
  def train():
      model.train()

      total_loss = 0

      for dat in train_loader:
         
          num_graphs = batch_size
          
          #dat.batch = None
          dat = dat.to(device)
          optimizer.zero_grad()

          outs = model(dat.x, dat.edge_index)
          
          loss = loss_op(outs[dat.mask], dat.y[dat.mask])
          temp = loss.item() * num_graphs
          total_loss += temp
          loss.backward()
          optimizer.step()
          
      return total_loss /2


  def test(loader):
      model.eval()

      ys, preds = [], []
      for dat in loader:
          ys.append(dat.y[dat.mask])
          with torch.no_grad():
              out = model(dat.x.to(device), dat.edge_index.to(device))[dat.mask]
          preds.append((out > 0).float().cpu())

      y, pred = torch.cat(ys, dim=0).numpy(), torch.cat(preds, dim=0).numpy()
      return f1_score(y, pred, average='micro') if pred.sum() > 0 else 0


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []

  print(model)
  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_f1 = test(val_loader)
      
      if early_stop:
          if stopper.step(val_f1, model):   
              break
            
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_f1 = test(test_loader)
  gcns.append(test_f1)
  print("Test Accuracy {:.4f}".format(test_f1))
    

Net(
  (conv1): GCNConv(50, 1024)
  (conv2): GCNConv(1024, 1024)
  (conv3): GCNConv(1024, 121)
)
Test Accuracy 0.6155
Net(
  (conv1): GCNConv(50, 1024)
  (conv2): GCNConv(1024, 1024)
  (conv3): GCNConv(1024, 121)
)
Test Accuracy 0.6106
Net(
  (conv1): GCNConv(50, 1024)
  (conv2): GCNConv(1024, 1024)
  (conv3): GCNConv(1024, 121)
)
Test Accuracy 0.6267
Net(
  (conv1): GCNConv(50, 1024)
  (conv2): GCNConv(1024, 1024)
  (conv3): GCNConv(1024, 121)
)
Test Accuracy 0.6299
Net(
  (conv1): GCNConv(50, 1024)
  (conv2): GCNConv(1024, 1024)
  (conv3): GCNConv(1024, 121)
)
Test Accuracy 0.6119
Net(
  (conv1): GCNConv(50, 1024)
  (conv2): GCNConv(1024, 1024)
  (conv3): GCNConv(1024, 121)
)
Test Accuracy 0.6171
Net(
  (conv1): GCNConv(50, 1024)
  (conv2): GCNConv(1024, 1024)
  (conv3): GCNConv(1024, 121)
)
Test Accuracy 0.6299
Net(
  (conv1): GCNConv(50, 1024)
  (conv2): GCNConv(1024, 1024)
  (conv3): GCNConv(1024, 121)
)
Test Accuracy 0.6199
Net(
  (conv1): GCNConv(50, 1024)
  (conv2): GCNConv(102

In [0]:
np.round(gcns,3).tolist()

[0.616, 0.611, 0.627, 0.63, 0.612, 0.617, 0.63, 0.62, 0.623, 0.627]

In [0]:
np.mean(gcns)

0.6211964104619412

## D_A_D_General/  DAD_w


In [0]:
import torch.nn as nn
from torch.nn import Linear
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import add_remaining_self_loops
from torch_scatter import scatter_add


class DADG_Conv(MessagePassing):
    def __init__(self, in_channels, out_channels, K=1, cached=False, bias=True,
                 improve=False, **kwargs):
        super(DADG_Conv, self).__init__(aggr='add', **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.K = K
        self.improve = improve
        self.cached = cached

        self.lin = Linear(in_channels, out_channels, bias=bias)
    

        self.reset_parameters()
        
    def reset_parameters(self):
        #gain = nn.init.calculate_gain('relu')
        
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_normal_(self.lin.weight, gain=gain)
        nn.init.zeros_(self.lin.bias)
        self.cached_result = None
        self.cached_num_edges = None
        
    '''def reset_parameters(self):
        self.lin.reset_parameters()
        self.cached_result = None
        self.cached_num_edges = None'''

    def forward(self, x, edge_index, edge_weight=None):
        """"""
        if self.cached and self.cached_result is not None:
            if edge_index.size(1) != self.cached_num_edges:
                raise RuntimeError(
                    'Cached {} number of edges, but found {}. Please '
                    'disable the caching behavior of this layer by removing '
                    'the `cached=True` argument in its constructor.'.format(
                        self.cached_num_edges, edge_index.size(1)))

        #if not self.cached:
            #x = self.lin(x)

        if not self.cached or self.cached_result is None:
            self.cached_num_edges = edge_index.size(1)
            edge_index, norm = self.My_norms(edge_index, x.size(0), edge_weight, self.improve
                                            , dtype=x.dtype)
            for k in range(self.K):
                #x = F.relu(self.propagate(edge_index, x=x, norm=norm))
                x = self.propagate(edge_index, x=x, norm=norm)
           
            #self.cached_result = x

        #if self.cached:
            #x = self.lin(self.cached_result)

        return x
    
    def My_norms(self, edge_index, num_nodes, edge_weight=None, improved=False,
             dtype=None):
        if edge_weight is None:
            edge_weight = torch.ones((edge_index.size(1), ), dtype=dtype,
                                     device=edge_index.device)

        fill_value = 1 if not improved else 2
        edge_index, edge_weight = add_remaining_self_loops(
            edge_index, edge_weight, fill_value, num_nodes)

        row, col = edge_index
        deg = scatter_add(edge_weight, row, dim=0, dim_size=num_nodes)
        deg_inv_sqrt = deg.pow(-0.5)
        #deg_inv_sqrt = deg.pow(-1)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0

        #return edge_index, deg_inv_sqrt[col] * edge_weight 
        return edge_index, deg_inv_sqrt[col] * edge_weight*deg_inv_sqrt[row]
    
    def message(self, x_j, norm):
        
        x_j = self.lin(x_j)
        return norm.view(-1, 1) * x_j

    def __repr__(self):
        return '{}({}, {}, K={})'.format(self.__class__.__name__,
                                         self.in_channels, self.out_channels,
                                         self.K)

In [0]:
h_dim = 256*4

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.lin1 = torch.nn.Linear(num_features, h_dim)
        
        self.conv1 = DADG_Conv(
            h_dim, h_dim, K=2, cached=True, improve=True)
        
        self.lin2 = torch.nn.Linear(h_dim, num_classes)
        
    def forward(self, x, edge_index):
        x = self.lin1(x)
        x = self.conv1(x, edge_index)
        x = self.lin2(x)
        return x
        

In [63]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
loss_op = BCEWithLogitsWrapper()
#optimizer = torch.optim.Adam(model.parameters(), lr=0.03)#0.009
optimizer = torch.optim.Adam(model.parameters(), lr=0.03, weight_decay=5e-6)


dads = []
for _ in range(10):
  batch_size = 2
  def train():
      model.train()

      total_loss = 0

      for dat in train_loader:
          
          num_graphs = batch_size
          
          #dat.batch = None
          dat = dat.to(device)
          optimizer.zero_grad()

          outs = model(dat.x, dat.edge_index)
          
          loss = loss_op(outs[dat.mask], dat.y[dat.mask])
          temp = loss.item() * num_graphs
          total_loss += temp
          loss.backward()
          optimizer.step()
          
      return total_loss /2


  def test(loader):
      model.eval()

      ys, preds = [], []
      for dat in loader:
          ys.append(dat.y[dat.mask])
          with torch.no_grad():
              out = model(dat.x.to(device), dat.edge_index.to(device))[dat.mask]
          preds.append((out > 0).float().cpu())

      y, pred = torch.cat(ys, dim=0).numpy(), torch.cat(preds, dim=0).numpy()
      return f1_score(y, pred, average='micro') if pred.sum() > 0 else 0


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []

  print(model)
  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_f1 = test(val_loader)
      
      if early_stop:
          if stopper.step(val_f1, model):   
              break
            
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_f1 = test(test_loader)
  dads.append(test_f1)
  print("Test Accuracy {:.4f}".format(test_f1))
    

Net(
  (lin1): Linear(in_features=50, out_features=1024, bias=True)
  (conv1): DADG_Conv(1024, 1024, K=5)
  (lin2): Linear(in_features=1024, out_features=121, bias=True)
)


KeyboardInterrupt: ignored

In [56]:
np.round(dads,3).tolist()

[0.536, 0.537, 0.542, 0.543, 0.543, 0.543, 0.543, 0.543, 0.544, 0.544]

In [57]:
np.mean(dads)

0.5420386204047024

In [58]:
np.mean([0.537, 0.532, 0.531, 0.538, 0.538, 0.539, 0.54, 0.543, 0.543, 0.569]
)

0.541

# D_A_D(SGC)/DAD_relu1


In [0]:
class P_GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels, K=1, cached=True, bias=True,
                 improve=False, **kwargs):
        super(P_GCNConv, self).__init__(aggr='add', **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.K = K
        self.improve = improve
        self.cached = cached

        #self.lin = Linear(in_channels, out_channels, bias=bias)

        self.reset_parameters()
        
    '''def reset_parameters(self):
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_uniform_(self.lin.weight, gain=gain)
        nn.init.zeros_(self.lin.bias)
        self.cached_result = None
        self.cached_num_edges = None'''

    '''def reset_parameters(self):
        uniform(self.in_channels, self.weight)
        uniform(self.in_channels, self.bias)'''
        
    def reset_parameters(self):
        #self.lin.reset_parameters()
        self.cached_result = None
        self.cached_num_edges = None

    def forward(self, x, edge_index, edge_weight=None):
        """"""
        if self.cached and self.cached_result is not None:
            if edge_index.size(1) != self.cached_num_edges:
                raise RuntimeError(
                    'Cached {} number of edges, but found {}. Please '
                    'disable the caching behavior of this layer by removing '
                    'the `cached=True` argument in its constructor.'.format(
                        self.cached_num_edges, edge_index.size(1)))

        

        if not self.cached or self.cached_result is None:
            self.cached_num_edges = edge_index.size(1)
            edge_index, norm = self.My_norms(edge_index, x.size(0), edge_weight, self.improve
                                            , dtype=x.dtype)
            x = F.relu(self.propagate(edge_index, x=x, norm=norm))

            for k in range(self.K-1):
                x = self.propagate(edge_index, x=x, norm=norm)
                
            #self.cached_result = x

        '''if self.cached:
            x = self.lin(self.cached_result)'''

        return x
    
    def My_norms(self, edge_index, num_nodes, edge_weight=None, improved=False,
             dtype=None):
        if edge_weight is None:
            edge_weight = torch.ones((edge_index.size(1), ), dtype=dtype,
                                     device=edge_index.device)

        fill_value = 1 if not improved else 2
        edge_index, edge_weight = add_remaining_self_loops(
            edge_index, edge_weight, fill_value, num_nodes)

        row, col = edge_index
        deg = scatter_add(edge_weight, row, dim=0, dim_size=num_nodes)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0

        return edge_index, deg_inv_sqrt[col] * edge_weight*deg_inv_sqrt[row] 
        
    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

    def __repr__(self):
        return '{}({}, {}, K={})'.format(self.__class__.__name__,
                                         self.in_channels, self.out_channels,
                                         self.K)

In [0]:
h_dim = 256*4

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.lin1 = torch.nn.Linear(num_features, h_dim)
        
        self.conv1 = P_GCNConv(
            h_dim, h_dim, K=2, cached=True, improve=True)
        
        self.lin2 = torch.nn.Linear(h_dim, num_classes)
        
    def forward(self, x, edge_index):
        x = self.lin1(x)
        x = self.conv1(x, edge_index)
        x = self.lin2(x)
        return x
        

In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
loss_op = BCEWithLogitsWrapper()
#optimizer = torch.optim.Adam(model.parameters(), lr=0.03)#0.009
optimizer = torch.optim.Adam(model.parameters(), lr=0.05, weight_decay=5e-6)

sgcs = []
for _ in range(10):
  batch_size = 2
  def train():
      model.train()

      total_loss = 0

      for dat in train_loader:
          
          num_graphs = batch_size
          
          #dat.batch = None
          dat = dat.to(device)
          optimizer.zero_grad()

          outs = model(dat.x, dat.edge_index)
          
          loss = loss_op(outs[dat.mask], dat.y[dat.mask])
          temp = loss.item() * num_graphs
          total_loss += temp
          loss.backward()
          optimizer.step()
          
      return total_loss /2


  def test(loader):
      model.eval()

      ys, preds = [], []
      for dat in loader:
          ys.append(dat.y[dat.mask])
          with torch.no_grad():
              out = model(dat.x.to(device), dat.edge_index.to(device))[dat.mask]
          preds.append((out > 0).float().cpu())

      y, pred = torch.cat(ys, dim=0).numpy(), torch.cat(preds, dim=0).numpy()
      return f1_score(y, pred, average='micro') if pred.sum() > 0 else 0


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []

  print(model)
  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_f1 = test(val_loader)
      
      if early_stop:
          if stopper.step(val_f1, model):   
              break
            
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_f1 = test(test_loader)
  sgcs.append(test_f1)
  print("Test Accuracy {:.4f}".format(test_f1))
    

Net(
  (lin1): Linear(in_features=50, out_features=1024, bias=True)
  (conv1): P_GCNConv(1024, 1024, K=2)
  (lin2): Linear(in_features=1024, out_features=121, bias=True)
)
Test Accuracy 0.6481
Net(
  (lin1): Linear(in_features=50, out_features=1024, bias=True)
  (conv1): P_GCNConv(1024, 1024, K=2)
  (lin2): Linear(in_features=1024, out_features=121, bias=True)
)
Test Accuracy 0.6483
Net(
  (lin1): Linear(in_features=50, out_features=1024, bias=True)
  (conv1): P_GCNConv(1024, 1024, K=2)
  (lin2): Linear(in_features=1024, out_features=121, bias=True)
)
Test Accuracy 0.6526
Net(
  (lin1): Linear(in_features=50, out_features=1024, bias=True)
  (conv1): P_GCNConv(1024, 1024, K=2)
  (lin2): Linear(in_features=1024, out_features=121, bias=True)
)
Test Accuracy 0.6394
Net(
  (lin1): Linear(in_features=50, out_features=1024, bias=True)
  (conv1): P_GCNConv(1024, 1024, K=2)
  (lin2): Linear(in_features=1024, out_features=121, bias=True)
)
Test Accuracy 0.6384
Net(
  (lin1): Linear(in_features=5

In [38]:
np.round(sgcs,3).tolist()

[0.648, 0.648, 0.653, 0.639, 0.638, 0.647, 0.644, 0.648, 0.647, 0.637]

In [39]:
np.mean(sgcs)

0.644992122991823

In [0]:
lr=0.005, weight_decay=5e-6 0.456

lr=0.05, weight_decay=5e-6 0.48, 0.46
lr=0.02, weight_decay=5e-6 0.47, 0.46
lr=0.01, weight_decay=5e-6  0.47, 0.46

lr=0.05, weight_decay=5e-4  0.45 0.47
lr=0.05, weight_decay=5e-2 0.46, 0.45 

lr=0.5, weight_decay=5e-6 0.4575 0.4533




# P_GAT_General！

In [0]:
import torch
import torch.nn as nn
from torch.nn import Linear
from torch.nn import Parameter
import torch.nn.functional as F
from torch_geometric.nn.inits import glorot, zeros
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax

class P_GATConv(MessagePassing):

    def __init__(self, in_channels, out_channels, k=2, negative_slope=0.2, dropout=0.6, bias=True, **kwargs):
        super(P_GATConv, self).__init__(aggr='add', **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.k = k
        self.negative_slope = negative_slope
        self.dropout = dropout
        self.bias = bias

        self.lin = Linear(in_channels, out_channels, bias=bias)
        self.att = Parameter(torch.Tensor(1, 2 * out_channels))
      
        self.reset_parameters()

    def reset_parameters(self):
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_normal_(self.lin.weight, gain=gain)
        nn.init.xavier_normal_(self.att, gain=gain)
        if self.bias:
            nn.init.zeros_(self.lin.bias)

    ''' def reset_parameters(self):
        self.lin.reset_parameters()
        #self.lin2.reset_parameters()
        glorot(self.att)'''
    
    def My_norms(self, x, edge_index, num_nodes):
        edge_index_j = edge_index[0]
        edge_index_i = edge_index[1]
        x_j = x[edge_index_j]
        x_i = x[edge_index_i]
        
        alpha0 = (torch.cat([x_i, x_j], dim=-1) * self.att).sum(dim=-1)
        alpha1 = (torch.cat([x_j, x_i], dim=-1) * self.att).sum(dim=-1)
        alpha = (alpha0 + alpha1)/2.0
        
        #alpha = (torch.cat([x_i, x_j], dim=-1) * self.att).sum(dim=-1)

        alpha = F.leaky_relu(alpha, self.negative_slope)

        
        alpha = softmax(alpha, edge_index_i, num_nodes)
        
        # Sample attention coefficients stochastically.
        #alpha = F.dropout(alpha, p=self.dropout, training=self.training)
        
        return alpha
    
    def forward(self, x, edge_index, size=None):
        """"""
        if size is None and torch.is_tensor(x):
            edge_index, _ = remove_self_loops(edge_index)
            edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        #x = self.lin(x)
        
        norm = self.My_norms(x, edge_index, x.size(0))
        # x 不放在relu里！！！
        for _ in range(self.k):

            x = F.relu(self.propagate(edge_index, size=size, x=x, norm=norm)) +x

        return x


    def message(self, x_j, norm):
        x_j = self.lin(x_j)
        return norm.view(-1, 1) * x_j
    
    def __repr__(self):
        return '{}({}, {}, heads={})'.format(self.__class__.__name__,
                                             self.in_channels,
                                             self.out_channels, self.k)

# P-GAT/P-GAT-relu1

In [0]:
import torch
import torch.nn as nn
from torch.nn import Linear
from torch.nn import Parameter
import torch.nn.functional as F
from torch_geometric.nn.inits import glorot, zeros
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax

class P_GAT(MessagePassing):

    def __init__(self, in_channels, out_channels, k=2, negative_slope=0.2, dropout=0.6, bias=True, improved=False,**kwargs):
        super(P_GAT, self).__init__(aggr='add', **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.k = k
        self.negative_slope = negative_slope
        self.dropout = dropout
        self.bias = bias
        self.improved = improved

        self.lin = Linear(in_channels, out_channels, bias=bias)
        self.att = Parameter(torch.Tensor(1, 2 * out_channels))
      
        self.reset_parameters()

    def reset_parameters(self):
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_normal_(self.lin.weight, gain=gain)
        nn.init.xavier_normal_(self.att, gain=gain)
        if self.bias:
            nn.init.zeros_(self.lin.bias)

    ''' def reset_parameters(self):
        self.lin.reset_parameters()
        #self.lin2.reset_parameters()
        glorot(self.att)'''
    
    def My_norms(self, x, edge_index, num_nodes, improved=False):
        edge_index_j = edge_index[0]
        edge_index_i = edge_index[1]
        x_j = x[edge_index_j]
        x_i = x[edge_index_i]
        
        alpha0 = (torch.cat([x_i, x_j], dim=-1) * self.att).sum(dim=-1)
        alpha1 = (torch.cat([x_j, x_i], dim=-1) * self.att).sum(dim=-1)
        alpha = (alpha0 + alpha1)/2.0

        if improved:
            self_weight = edge_index_i== edge_index_j
            alpha[self_weight] += 1.0
        
        #alpha = (torch.cat([x_i, x_j], dim=-1) * self.att).sum(dim=-1)

        alpha = F.leaky_relu(alpha, self.negative_slope)

        
        alpha = softmax(alpha, edge_index_i, num_nodes)
        
        # Sample attention coefficients stochastically.
        #alpha = F.dropout(alpha, p=self.dropout, training=self.training)
        
        return alpha
    
    def forward(self, x, edge_index, size=None):
        """"""
        if size is None and torch.is_tensor(x):
            edge_index, _ = remove_self_loops(edge_index)
            edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        x = self.lin(x)
        
        norm = self.My_norms(x, edge_index, x.size(0))
            

        
        x = F.relu(self.propagate(edge_index, size=size, x=x, norm=norm)) +x

        for _ in range(self.k-1):

            x = self.propagate(edge_index, size=size, x=x, norm=norm) + x

        return x


    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j
    
    def __repr__(self):
        return '{}({}, {}, heads={})'.format(self.__class__.__name__,
                                             self.in_channels,
                                             self.out_channels, self.k)

In [0]:
#h_dim =  2 * 256

In [0]:
h_dim =  4 * 256

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.lin1 = torch.nn.Linear(num_features, h_dim)
        
        self.conv1 = P_GAT(
             h_dim, h_dim, k=2)
        
        self.lin2 = torch.nn.Linear(h_dim, num_classes)
        
    def forward(self, x, edge_index):
        x = self.lin1(x)
        x = self.conv1(x, edge_index)
        x = self.lin2(x)
        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
loss_op = BCEWithLogitsWrapper()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)#0.002
#optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-6)

In [87]:

gat_g_s = []
for _ in range(10):
  batch_size = 2
  def train():
      model.train()

      total_loss = 0

      for dat in train_loader:
          
          num_graphs = batch_size
          
          #dat.batch = None
          dat = dat.to(device)
          optimizer.zero_grad()

          outs = model(dat.x, dat.edge_index)
          
          loss = loss_op(outs[dat.mask], dat.y[dat.mask])
          temp = loss.item() * num_graphs
          total_loss += temp
          loss.backward()
          optimizer.step()
          
      return total_loss /2


  def test(loader):
      model.eval()

      ys, preds = [], []
      for dat in loader:
          ys.append(dat.y[dat.mask])
          with torch.no_grad():
              out = model(dat.x.to(device), dat.edge_index.to(device))[dat.mask]
          preds.append((out > 0).float().cpu())

      y, pred = torch.cat(ys, dim=0).numpy(), torch.cat(preds, dim=0).numpy()
      return f1_score(y, pred, average='micro') if pred.sum() > 0 else 0


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []

  print(model)
  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_f1 = test(val_loader)
      
      if early_stop:
          if stopper.step(val_f1, model):   
              break
            
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_f1 = test(test_loader)
  gat_g_s.append(test_f1)
  print("Test Accuracy {:.4f}".format(test_f1))
    

Net(
  (lin1): Linear(in_features=50, out_features=1024, bias=True)
  (conv1): P_GAT(1024, 1024, heads=2)
  (lin2): Linear(in_features=1024, out_features=121, bias=True)
)
Test Accuracy 0.4932
Net(
  (lin1): Linear(in_features=50, out_features=1024, bias=True)
  (conv1): P_GAT(1024, 1024, heads=2)
  (lin2): Linear(in_features=1024, out_features=121, bias=True)
)
Test Accuracy 0.4978
Net(
  (lin1): Linear(in_features=50, out_features=1024, bias=True)
  (conv1): P_GAT(1024, 1024, heads=2)
  (lin2): Linear(in_features=1024, out_features=121, bias=True)
)
Test Accuracy 0.5186
Net(
  (lin1): Linear(in_features=50, out_features=1024, bias=True)
  (conv1): P_GAT(1024, 1024, heads=2)
  (lin2): Linear(in_features=1024, out_features=121, bias=True)
)
Test Accuracy 0.5155
Net(
  (lin1): Linear(in_features=50, out_features=1024, bias=True)
  (conv1): P_GAT(1024, 1024, heads=2)
  (lin2): Linear(in_features=1024, out_features=121, bias=True)
)
Test Accuracy 0.5159
Net(
  (lin1): Linear(in_features=5

In [88]:
np.round(gat_g_s,3).tolist()

[0.493, 0.498, 0.519, 0.516, 0.516, 0.515, 0.526, 0.533, 0.537, 0.524]

In [89]:
np.mean(gat_g_s)

0.5176159163261744

In [90]:
10

10